<a href="https://colab.research.google.com/github/joaopaiva/trading-strategies/blob/main/LSTM%20Model%20to%20Predict%20Stock%20Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# suppress output
%%capture
!pip install yfinance pandas_ta

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, date, time, timedelta
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
yf.pdr_override()

# Define the ticker symbol and start and end dates
stocksymbols = ['BNB-USD']
# start_date = '2022-01-01'
# end_date = '2022-02-18'
start_date = datetime.today() - timedelta(days=6)
start_date = start_date.strftime("%Y-%m-%d")
end_date = datetime.today()

def getHistoricalData(stocks = stocksymbols ,start = start_date , end = end_date):
    # Get the data for the ticker
    ticker_data = yf.Ticker(stocksymbols[0])

    # Get the historical prices for the ticker
    data = ticker_data.history(start=start_date, end=end_date, interval='1m')

    # data = web.get_data_yahoo(stocks , data_source='yahoo' , start = start ,end= end )
    return data

In [ ]:
async def create_ltsm_model(df):
  # Preprocess the data
  data = df.filter(['Close']).values
  training_data_len = int(np.ceil(len(data) * 0.8))
  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_data = scaler.fit_transform(data)

  # Create the training data set
  train_data = scaled_data[0:training_data_len, :]
  x_train = []
  y_train = []
  for i in range(60, len(train_data)):
      x_train.append(train_data[i-60:i, 0])
      y_train.append(train_data[i, 0])
  x_train, y_train = np.array(x_train), np.array(y_train)
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

  # Build the LSTM model
  model = Sequential()
  model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
  model.add(LSTM(50, return_sequences=False))
  model.add(Dense(25))
  model.add(Dense(1))

  # Compile the model
  model.compile(optimizer='adam', loss='mean_squared_error')

  # Train the model
  model.fit(x_train, y_train, batch_size=1, epochs=1)

  # Create the testing data set
  test_data = scaled_data[training_data_len - 60:, :]
  x_test = []
  y_test = data[training_data_len:, :]
  for i in range(60, len(test_data)):
      x_test.append(test_data[i-60:i, 0])
  x_test = np.array(x_test)
  x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  # Make predictions
  predictions = model.predict(x_test)
  predictions = scaler.inverse_transform(predictions)


In [ ]:
# Load historical stock data
data = getHistoricalData(stocksymbols)